In [16]:

ch_ideal = ["PZ", "PO3", "PO5", "PO4", "PO6", "POZ", "O1", "OZ", "O2"] #canais ideais
ch_names = ["FP1", "FPZ", "FP2", "AF3", "AF4", "F7", "F5", "F3", "F1", "FZ", "F2", "F4", "F6", "F8", "FT7", "FC5", "FC3", "FC1", "FCZ", "FC2", "FC4", "FC6", "FT8", "T7", "C5", "C3", "C1", "CZ", "C2", "C4", "C6", "T8", "M1", "TP7", "CP5", "CP3", "CP1", "CPZ", "CP2", "CP4", "CP6", "TP8", "M2", "P7", "P5", "P3", "P1", "PZ", "P2", "P4", "P6", "P8", "PO7", "PO5", "PO3", "POZ", "PO4", "PO6", "PO8", "CB1", "O1", "OZ", "O2", "CB2"]


import mne
import numpy as np
dataset1 = np.load("../../datasets/beta/data1_filtered.npy")
dataset2 = np.load("../../datasets/beta/data2_filtered.npy")

labels1 = np.load("../../datasets/beta/labels1.npy")
labels2 = np.load("../../datasets/beta/labels2.npy")
# entendimento dos dados
dataset1.shape, dataset2.shape, labels1.shape, labels2.shape

((160, 64, 80), (160, 64, 80), (160,), (160,))

In [17]:
#keep only the ideal channels
channel_name_to_index = {ch_name: index for index, ch_name in enumerate(ch_names)}

# Get the indices of the ideal channels
ideal_channel_indices = [channel_name_to_index[ch_name] for ch_name in ch_ideal]

# Keep only the ideal channels in the datasets
dataset1 = dataset1[:, ideal_channel_indices, :]
dataset2 = dataset2[:, ideal_channel_indices, :]

dataset1.shape, dataset2.shape


((160, 9, 80), (160, 9, 80))

In [18]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
dataset1 = dataset1.reshape(dataset1.shape[0], dataset1.shape[1] * dataset1.shape[2])
dataset2 = dataset2.reshape(dataset2.shape[0], dataset2.shape[1] * dataset2.shape[2])


X1 = StandardScaler().fit_transform(dataset1)
y1 = LabelEncoder().fit_transform(labels1)

X2 = StandardScaler().fit_transform(dataset2)
y2 = LabelEncoder().fit_transform(labels2)

X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42)

svc =  SVC(kernel='linear', C=1, random_state=42, probability=True)
svc.fit(X_train1, y_train1)
pred1 = svc.predict(X_test1)

svc =  SVC(kernel='linear', C=1, random_state=42, probability=True)
svc.fit(X_train2, y_train2)
pred2 = svc.predict(X_test2)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score

print("Dataset 1")
print("acuracia: %.2f" % (accuracy_score(y_test1, pred1)* 100),"%")
print("f1-score: %.2f"% (f1_score(y_test1, pred1, average='macro')*100),"%")

print("Dataset 2")
print("acuracia: %.2f" % (accuracy_score(y_test2, pred2)* 100),"%")
print("f1-score:  %.2f" % (f1_score(y_test2, pred2, average='macro')* 100),"%")




Dataset 1
acuracia: 0.00 %
f1-score: 0.00 %
Dataset 2
acuracia: 31.25 %
f1-score:  26.89 %


## Classificação utilizando RFE para seleção de atributos e SVM para classificação

In [19]:

from sklearn.feature_selection import RFECV


dataset1 = np.load("../../datasets/beta/data1_filtered.npy")
dataset2 = np.load("../../datasets/beta/data2_filtered.npy")


dataset1 = dataset1.reshape(dataset1.shape[0], dataset1.shape[1] * dataset1.shape[2])
dataset2 = dataset2.reshape(dataset2.shape[0], dataset2.shape[1] * dataset2.shape[2])

X1 = StandardScaler().fit_transform(dataset1)
y1 = LabelEncoder().fit_transform(labels1)

X2 = StandardScaler().fit_transform(dataset2)
y2 = LabelEncoder().fit_transform(labels2)

svc = SVC(kernel="linear")
rfe = RFECV(svc, step=0.1, min_features_to_select=20, cv=3)
X1 = rfe.fit_transform(X1, y1)
X2 = rfe.fit_transform(X2, y2)

X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=42)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42)


svc =  SVC(kernel='linear', C=1, random_state=42, probability=True)
svc.fit(X_train1, y_train1)
pred1 = svc.predict(X_test1)

svc =  SVC(kernel='linear', C=1, random_state=42, probability=True)
svc.fit(X_train2, y_train2)
pred2 = svc.predict(X_test2)

from sklearn.metrics import accuracy_score, f1_score

print("Dataset 1")
print("acuracia: %.2f" % (accuracy_score(y_test1, pred1)* 100),"%")
print("f1-score: %.2f"% (f1_score(y_test1, pred1, average='macro')*100),"%")

print("Dataset 2")
print("acuracia: %.2f" % (accuracy_score(y_test2, pred2)* 100),"%")
print("f1-score:  %.2f" % (f1_score(y_test2, pred2, average='macro')* 100),"%")


Dataset 1
acuracia: 21.88 %
f1-score: 18.14 %
Dataset 2
acuracia: 50.00 %
f1-score:  45.56 %
